# HW 2: Language Modeling

In this homework you will be building several varieties of language models.

## Goal

We ask that you construct the following models in PyTorch:

1. A trigram model with linear-interpolation. $$p(y_t | y_{1:t-1}) =  \alpha_1 p(y_t | y_{t-2}, y_{t-1}) + \alpha_2 p(y_t | y_{t-1}) + (1 - \alpha_1 - \alpha_2) p(y_t) $$
2. A neural network language model (consult *A Neural Probabilistic Language Model* http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)
3. An LSTM language model (consult *Recurrent Neural Network Regularization*, https://arxiv.org/pdf/1409.2329.pdf) 
4. Your own extensions to these models...


Consult the papers provided for hyperparameters.

 


## Setup

This notebook provides a working definition of the setup of the problem itself. You may construct your models inline or use an external setup (preferred) to build your system.

In [ ]:
# Text text processing library
import torchtext
from torchtext.vocab import Vectors

The dataset we will use of this problem is known as the Penn Treebank (http://aclweb.org/anthology/J93-2004). It is the most famous dataset in NLP and includes a large set of different types of annotations. We will be using it here in a simple case as just a language modeling dataset.

To start, `torchtext` requires that we define a mapping from the raw text data to featurized indices. These fields make it easy to map back and forth between readable data and math, which helps for debugging.

In [ ]:
# Our input $x$
TEXT = torchtext.data.Field()

Next we input our data. Here we will use the first 10k sentences of the standard PTB language modeling split, and tell it the fields.

In [ ]:
# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)

The data format for language modeling is strange. We pretend the entire corpus is one long sentence.

In [ ]:
print('len(train)', len(train))

Here's the vocab itself. (This dataset has unk symbols already, but torchtext adds its own.)

In [ ]:
TEXT.build_vocab(train)
print('len(TEXT.vocab)', len(TEXT.vocab))

When debugging you may want to use a smaller vocab size. This will run much faster.

In [ ]:
if False:
    TEXT.build_vocab(train, max_size=1000)
    len(TEXT.vocab)

The batching is done in a strange way for language modeling. Each element of the batch consists of `bptt_len` words in order. This makes it easy to run recurrent models like RNNs. 

In [ ]:
train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=10, device=-1, bptt_len=32, repeat=False)

Here's what these batches look like. Each is a string of length 32. Sentences are ended with a special `<eos>` token.

In [ ]:
it = iter(train_iter)
batch = next(it) 
print(vars(batch))
print("Size of text batch [max bptt length, batch size]", batch.text.size())
print("Second in batch", batch.text[:, 2])
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text[:, 2].data]))

The next batch will be the continuation of the previous. This is helpful for running recurrent neural networks where you remember the current state when transitioning.

In [ ]:
batch = next(it)
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text[:, 2].data]))
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text[:, 3].data]))
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text[:, 4].data]))

There are no separate labels. But you can just use an offset `batch.text[1:]` to get the next word.

## Assignment

Now it is your turn to build the models described at the top of the assignment. 

Using the data given by this iterator, you should construct 3 different torch models that take in batch.text and produce a distribution over the next word. 

When a model is trained, use the following test function to produce predictions, and then upload to the kaggle competition: https://www.kaggle.com/c/cs287-hw2-s18

For the final Kaggle test, we will have you do a next word prediction task. We will provide a 10 word prefix of sentences, and it is your job to predict 10 possible next word candidates

In [ ]:
!head input.txt

As a sample Kaggle submission, let us build a simple unigram model.  

In [ ]:
from collections import Counter
count = Counter()
for b in iter(train_iter):
    count.update(b.text.view(-1).data.tolist())
count[TEXT.vocab.stoi["<eos>"]] = 0
predictions = [TEXT.vocab.itos[i] for i, c in count.most_common(20)]
print(predictions)
with open("sample.txt", "w") as fout: 
    print("id,word", file=fout)
    for i, l in enumerate(open("input.txt"), 1):
        print("%d,%s"%(i, " ".join(predictions)), file=fout)


In [ ]:
!head sample.txt

The metric we are using is mean average precision of your 20-best list. 

$$MAP@20 = \frac{1}{|D|} \sum_{u=1}^{|D|} \sum_{k=1}^{20} Precision(u, 1:k)$$

Ideally we would use log-likelihood or ppl as discussed in class, but this is the best Kaggle gives us. This takes into account whether you got the right answer and how highly you ranked it. 

In particular, we ask that you do not game this metric. Please submit *exactly 20* unique predictions for each example.


As always you should put up a 5-6 page write-up following the template provided in the repository:  https://github.com/harvard-ml-courses/cs287-s18/blob/master/template/

# Neural Probabalistic Language Model

In [1]:
# Text text processing library
import torchtext
import torch
import math 
import torch.nn as nn 
from torchtext.vocab import Vectors
DEBUG = True 
# Our input $x$
TEXT = torchtext.data.Field()
# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)
TEXT.build_vocab(train)
print('len(TEXT.vocab)', len(TEXT.vocab))

if DEBUG == True:
    TEXT.build_vocab(train, max_size=1000)
    len(TEXT.vocab)
    print('len(TEXT.vocab)', len(TEXT.vocab))
    
train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=12, device=-1, bptt_len=32, repeat=False, shuffle=False)

len(TEXT.vocab) 10001
len(TEXT.vocab) 1002


In [58]:
class LSTM(nn.Module):
    
    def __init__(self, V_vocab_dim, M_embed_dim, H_hidden_dim, N_seq_len, B_batch_size):
        super(LSTM, self).__init__()
        self.batch_size = B_batch_size
        self.hidden_dim = H_hidden_dim
        self.vocab_dim = V_vocab_dim
        self.embed = nn.Embedding(V_vocab_dim, M_embed_dim)
        self.lstm = nn.LSTM(M_embed_dim, self.hidden_dim, dropout=0.5)
        self.fc = nn.Linear(self.hidden_dim, V_vocab_dim)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        if torch.cuda.is_available():
            return (torch.autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)).cuda(),
                torch.autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)).cuda())
        else:   
            return (torch.autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)),
                torch.autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))

    def forward(self, sentence):
        #print(sentence.shape)
        #input size N_seq_len x B_batch_size
        embeds = self.embed(sentence)
        #print(embeds.shape)
        # embeds size N_seq_len x B_batch_size x M_embed_dim
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        #print(lstm_out.shape)
        # lstm_out N_seq_len x B_batch_size x H_hidden_dim
        out = self.fc(lstm_out)
        #print(out.shape)
        # out N_seq_len x B_batch_size x V_vocab_dim
        return out

In [50]:
def evaluate(model, data_iterator):
    # Turn on evaluation mode which disables dropout.
    #model.eval()
    total_loss = 0
    batch_count = 0
    for batch in iter(data_iterator):
        model.hidden = model.init_hidden()
        output = model(batch.text)
        batch_loss = criterion(output.view(-1, model.vocab_dim), batch.target.view(-1)).data
        total_loss += batch_loss
        batch_count += 1
    return total_loss[0] / batch_count

def train_batch(model, criterion, optim, batch, target):
    # initialize hidden vectors
    model.zero_grad()
    model.hidden = model.init_hidden()
    # calculate forward pass
    y = model(batch)
    # calculate loss
    loss = criterion(y.view(-1, model.vocab_dim), target.view(-1))
    # backpropagate and step
    loss.backward()
    torch.nn.utils.clip_grad_norm(model.parameters(), 0.25)
    optim.step()
    return loss.data[0]

# training loop
def run_training(model, criterion, optim, data_iterator, val_iter):

    for e in range(n_epochs):
        batches = 0
        epoch_loss = 0
        for batch in iter(data_iterator):
            batch_loss = train_batch(model, criterion, optim, batch.text, batch.target)
            batches += 1
            epoch_loss += batch_loss
        epoch_loss /= batches
        print("Epoch ", e, " Loss: ", epoch_loss, "Perplexity: ", math.exp(epoch_loss))
        train_loss = evaluate(model, data_iterator)
        print("Epoch Train Loss: ", train_loss, "Perplexity: ", math.exp(train_loss))
        val_loss = evaluate(model, val_iter)
        print("Epoch Val Loss: ", val_loss, "Perplexity: ", math.exp(val_loss))
        torch.save(model.state_dict(), 'LSTM_small_model.pt')

In [51]:
# size of the embeddings and vectors
n_embedding = 30
n_hidden = 30
seq_len = 32
batch_size = 12

# initialize LSTM
lstm_model = LSTM(len(TEXT.vocab), n_embedding, n_hidden, seq_len, batch_size)

n_epochs = 10
learning_rate = .1
criterion = nn.CrossEntropyLoss()
optim = torch.optim.SGD(lstm_model.parameters(), lr = learning_rate)


run_training(lstm_model, criterion, optim, train_iter, val_iter)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: generator 'BPTTIterator.__iter__' raised StopIteration


Epoch  0  Loss:  4.731864180165054 Perplexity:  113.5069626717025


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'BPTTIterator.__iter__' raised StopIteration
  


Epoch Train Loss:  4.490187987272821 Perplexity:  89.13820115105892
Epoch Val Loss:  4.533652824441386 Perplexity:  93.09801141998173
Epoch  1  Loss:  4.436292855023648 Perplexity:  84.46125048329468
Epoch Train Loss:  4.395524350733168 Perplexity:  81.0871377175898
Epoch Val Loss:  4.440269351623219 Perplexity:  84.79777901747588
Epoch  2  Loss:  4.358900011693678 Perplexity:  78.17109981753761
Epoch Train Loss:  4.329859900996489 Perplexity:  75.93364759558882
Epoch Val Loss:  4.375060086423251 Perplexity:  79.44461295149048
Epoch  3  Loss:  4.3009359284109046 Perplexity:  73.76880371957871
Epoch Train Loss:  4.278217452757125 Perplexity:  72.11178271308589
Epoch Val Loss:  4.323572662827882 Perplexity:  75.45773233216681
Epoch  4  Loss:  4.2542598095869835 Perplexity:  70.40468502406233
Epoch Train Loss:  4.236350310434737 Perplexity:  69.15499642870215
Epoch Val Loss:  4.2816620663657705 Perplexity:  72.36060824904303
Epoch  5  Loss:  4.216132522072491 Perplexity:  67.7708744368326

In [70]:
lstm_model = LSTM(len(TEXT.vocab), n_embedding, n_hidden, seq_len, batch_size)
lstm_model.load_state_dict(torch.load('LSTM_small_model.pt'))
with open("sample.txt", "w") as fout: 
    print("id,word", file=fout)
    for i, l in enumerate(open("input.txt"), 1):
        print(l)
        input_tokens = l.split(" ")[:-1]
        input_index = torch.LongTensor([TEXT.vocab.stoi[t] for t in input_tokens]).unsqueeze(1)
        lstm_model.hidden = lstm_model.init_hidden()
        output = lstm_model(torch.autograd.Variable(input_index))
        clean_output = output[-1, 0, :].view(-1, lstm_model.vocab_dim)
        max_values, max_indices = torch.topk(clean_output, 20)
        print(" ".join([TEXT.vocab.itos[int(i)] for i in max_indices.data[0, :]]))

        #print("%d,%s"%(i, " ".join(predictions)), file=fout)

but while the new york stock exchange did n't fall ___

<unk> a the to N <eos> in an n't its that for and $ as it on at about by
some circuit breakers installed after the october N crash failed ___

<eos> to a <unk> of N in and for million from the $ on or than billion by at as
the N stock specialist firms on the big board floor ___

<unk> to <eos> in the a and of by that for on from with at said as 's N is
big investment banks refused to step up to the plate ___

<unk> <eos> of and 's in to for is market said that the on from with a by or which
heavy selling of standard & poor 's 500-stock index futures ___

<unk> the of N to a in <eos> $ and as for that at from it its on by about
seven big board stocks ual amr bankamerica walt disney capital ___

<unk> <eos> of and to in the for as that from it a by at on which or about said
once again the specialists were not able to handle the ___

<unk> company u.s. new market N first stock government past federal state $ same year big two house w

<unk> <eos> and of to N in 's from on is the or that for said a was will at
the <unk> between the stock and futures markets friday will ___

<unk> be a the have N been $ year n't to of <eos> 's do is for quarter and are
the big board 's mr. <unk> said our <unk> performance ___

<eos> <unk> and in the of it 's for to is that a with said on from was at has
but the exchange will look at the performance of all ___

<unk> the N $ a <eos> in and of from at year for it or on to shares its 's
obviously we 'll take a close look at any situation ___

<unk> of to <eos> 's a said and in for the is have that as was are has be n't
see related story fed ready to <unk> big funds wsj ___

<unk> <eos> to and in the of a for by that on with as from N 's at said is
but specialists complain privately that just as in the N ___

<unk> <eos> N and 's of to in for million from or is market shares billion years was a said
mr. phelan said it will take another day or two ___

<unk> <eos> of the in to a and as for

<unk> a n't york the said to N $ of not and that are an in also have as stock
he added that the company <unk> the union 's resolve ___

<unk> <eos> of to and in for the a 's that on by from with said as at is will
in separate developments talks have broken off between machinists representatives ___

<unk> <eos> of and to in for the a by that 's on N with from as said at is
the union is continuing to work through its expired contract ___

<unk> <eos> of to in and for than as that said a n't 's by is the co. with on
it had planned a strike vote for next sunday but ___

<unk> the <eos> of to and that it in as a for he 's said they its mr. at is
united auto workers local N which represents N workers at ___

N the a <unk> $ its an about this new than in million more their his that to stock years
and boeing on friday said it received an order from ___

the <unk> $ a N its an it <eos> in this new that about their mr. one and for as
the planes long range versions of the <unk> <unk> will ___

<

<unk> the be a $ N have its make <eos> do it buy sell their an get new take his
but the numbers become <unk> and subject to <unk> between ___

<unk> the N a in <eos> its and $ to that an mr. for it but at on by this
you say you could have sold x percent of this ___

<unk> year <eos> of the and 's N is to week in new a it for or have are on
quantum hopes the morris plant where limited production got under ___

<unk> the a $ to <eos> N its as of an in about and that for at their more than
the plant usually accounts for N N to N N ___

<eos> to <unk> million in of N and from billion a cents for or on shares at by share with
the plant expansion should strengthen the company 's <unk> in ___

the <unk> a N its new an their his this it $ that <eos> other mr. some one and all
by lifting <unk> production the expansion will also lower the ___

<unk> company u.s. new market N first stock government past federal state $ same year big house quarter two world
quantum is also tightening its grip on i

<unk> N the <eos> a and of to 's that as is for said in it mr. are at with
when attack hit the shelves in N p&g 's share ___

<eos> <unk> of in and to the for from a at or as on that it by 's with said
with the help of <unk> <unk> p&g 's share is ___

<unk> n't a the to <eos> in been not that an and of as for york it its have $
while the japanese have embraced the compact packaging and convenience ___

<unk> <eos> and of to in the 's for that by a on as with said is at from who
the company may have chosen to market the product under ___

<unk> the <eos> a $ of its in to and it as that for at about an this their from
with <unk> however it is n't always easy to persuade ___

<unk> the <eos> a to and in by for that on its as N with of at it from an
in the early 1980s p&g tried to launch here a ___

<unk> N year $ new share company stock than u.s. market first time federal week major <eos> the government big
but the product which was n't as concentrated as the ___

<unk> company u.s. new N

...

<unk> the a $ N its <eos> as about an to in their of and more this at it for
freeport-mcmoran resource partners as owner of the <unk> technology receives ___

<unk> <eos> to and in of the a for by that on with as 's from at said is it
business week subscribers may hear this week 's issue talking ___

<unk> <eos> in and to of the a for by on that from as with at 's said N or
a <unk> ad from texas instruments inc. running in approximately ___

<unk> <eos> and to the of in a for 's that by on with as from said at is which
readers who pull off a piece of tape and press ___

<unk> <eos> the to and in a for N of that with as by it mr. on at from its
the talking chip is n't cheap the <unk> cost to ___

<unk> be the a $ N have <eos> its make buy do sell it their an get take new more
previous efforts have included musical ads featuring simple <unk> chips ___

<unk> <eos> and to in the of a for by that on with as 's at from said is who
and for those who miss the message the first time ___

<eos>

of all the one-time expenses incurred by a corporation or ___

<unk> the N a of $ <eos> to in and as at for its that an from on about more
to take full advantage of the financial opportunities in this ___

<unk> <eos> year of and the 's N to is week in or new for market a which have it
it must also evaluate the real-estate market in the chosen ___

<unk> <eos> of and 's in to for market said is the that a on from which will with or
specifically it must understand how real-estate markets <unk> to shifts ___

<unk> the <eos> a to and in for by that on its as N of with from at an it
when a regional economy <unk> cold the local real-estate market ___

<unk> <eos> to in and for said 's of is from on with will that by has are was as
in other words real-estate market indicators such as building permits ___

<unk> <eos> and of to in for the by that a on 's with as from said at is who
this was seen in the late 1960s in los angeles ___

<unk> the <eos> a N to that in it and an its for as said mr

<unk> of <eos> and 's to in is for said that the market on from was by with as will
it said the N rail cars are in addition to ___

<unk> the N $ a be its have <eos> buy make new an do sell it their get take more
when china opened its doors to foreign investors in N ___

<eos> million N <unk> billion to and a of in the for from or 's cents years on with at
today with about N N of the companies ' products ___

<unk> <eos> N $ to in the a from of and on for that at year by as or it
the toy makers were pushed across the border by rising ___

<unk> <eos> to the and in of a for by that on as with 's from at N said its
but in the wake of the <unk> in beijing on ___

the <unk> a <eos> its it and that N an of mr. in to their as his this he at
although the manufacturers stress that production has n't been affected ___

<unk> <eos> to and in the a of for by that on as with at from 's it said its
the toy makers and their foreign buyers cite uncertainty about ___

N <unk> the $ a <eos> to and in o

<unk> company u.s. N new market first stock $ government past year federal same big state two third quarter sale
but recent developments have made the networks and nbc president ___

<unk> of <eos> and 's to a for the in said is that with which as has was on will
the latest <unk> sony corp. 's plan to acquire columbia ___

<unk> <eos> to in and for of said that 's is with as it on will from has are at
i wonder what walter <unk> will think of the <unk> ___

<unk> of <eos> and 's in to is said for that the market was on has will with which by
in his article mr. <unk> who was in europe last ___

<unk> year <eos> of new and the market week stock u.s. for company in quarter which that a trading 's
he did n't mention that nbc in the meantime was ___

<unk> the a <eos> to in and of that an for as N n't its at than on it said
an nbc spokesman <unk> that mr. <unk> 's <unk> was ___

<unk> a the to <eos> in n't an that and as for of its not it N been by at
the big three 's <unk> for deregulation 

we want to see nelson <unk> and all our <unk> ___

<unk> <eos> of and in to 's for the that on by is as said a from with at who
